In [3]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

def set_volume(v):
    # Convert the input value into a float between -65.25 and 0
    decibels = -65.25 + v * (65.25 / 100)
    volume.SetMasterVolumeLevel(decibels, None)

# Ask for a volume level from the user
v = float(input("Enter a volume level (0-100): "))
set_volume(v)


Enter a volume level (0-100):  100


In [6]:
display(volume_slider)

IntSlider(value=0, description='Volume:')

In [7]:
import cv2
import mediapipe as mp
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize video capture
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Unable to open camera")
    exit()

# Initialize the volume controller
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Function to set the volume
def set_volume(v):
    decibels = -65.25 + v * (65.25 / 100)
    volume.SetMasterVolumeLevel(decibels, None)

# Initialize Mediapipe Hands object
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    while True:
        # Read the frame from the video capture
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame from camera")
            break

        # Convert the frame to RGB for Mediapipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with Mediapipe
        results = hands.process(frame_rgb)

        # Draw hand landmarks on the frame
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))

                # Get the landmarks for the thumb and index finger
                thumb_tip = np.array([hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y])
                index_finger_tip = np.array([hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y])

                # Calculate the distance between the thumb and index finger
                distance = np.linalg.norm(thumb_tip - index_finger_tip)

                # Map the distance to a volume level and set the volume
                volume_level = np.interp(distance, [0, 0.5], [0, 100])  # Adjust the range as needed
                set_volume(volume_level)

        # Display the frame
        cv2.imshow("Hand Detection", frame)

        # Check for the 'q' key to exit
        if cv2.waitKey(1) == ord("q"):
            break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
